Stand: in Entwicklung
TODO
- die Api schnittstellen fehlen
- Datenaufbereitung
- Symbole hochladen

In [1]:
import sys
import os
import pandas as pd
from datetime import datetime

# Pfad zum Modul: /home/ageq/Git_Projects/MLdatalake/customs
current_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', 'customs')))

from Module.SQLAlchemy_functions import start_session, Symbol, MinuteBar

#/home/ageq/Git_Projects/MLdatalake/customs/Module/Grabber/Yahoofinance.py

from Module.Grabber.Yahoofinance import load_yf_data


#session = start_session()

In [2]:
# Neues Asset hinzufügen
def add_new_asset(session, row):
    try:
        asset = Symbol(
            ticker=row['ticker'],
            name=row['name'],
            market=row['market'],
            active=row['active']
        )
        session.add(asset)
        session.commit()
        print(f"Asset {row['ticker']} erfolgreich hinzugefügt.")
    except Exception as e:
        print(f"Fehler beim Hinzufügen des Assets {row['ticker']}: {e}")


In [3]:
# Aktualisieren der letzten Daten in der Asset-Tabelle
def update_asset_last_update(session, ticker, new_last_update):
    try:
        asset = check_if_asset_exists(session, ticker)
        if asset:
            asset.last_update = datetime.strptime(new_last_update, '%Y-%m-%d')
            session.commit()
            print(f"Asset {ticker} wurde erfolgreich aktualisiert.")
        else:
            print(f"Asset {ticker} nicht gefunden.")
    except Exception as e:
        print(f"Fehler beim Aktualisieren des Assets {ticker}: {e}")

#TODO bezeichung ändern
# Prüfen, ob das Asset existiert
def check_if_asset_exists(session, ticker):
    return session.query(Symbol).filter_by(ticker=ticker).first()



In [4]:
# Daten in MinuteBar-Tabelle einfügen
# und in die Datenbank einfügen
def insert_minute_bar(session, data, symbol_id):
    try:
        for index, row in data.iterrows():
            minute_bar = MinuteBar(
                date=row['date'],
                open=row['open'],
                high=row['high'],
                low=row['low'],
                close=row['close'],
                volume=row['volume'],
                symbol_id=symbol_id
            )
            session.add(minute_bar)
        session.commit()
        print(f"Minutenbalken für Symbol-ID {symbol_id} erfolgreich eingefügt.")
    except Exception as e:
        print(f"Fehler beim Einfügen der MinuteBars für Symbol-ID {symbol_id}: {e}")
        
# Daten für 5-Minuten und 30-Minuten-Bars sind ähnlich:
# insert_five_minute_bar, insert_thirty_minute_bar (folgt ähnlicher Logik)


In [5]:

# Dummy-Funktionen für verschiedene Exchanges
def process_binance_asset(session, row):
    print("Binance ist noch nicht implementiert.")
    #print(f"Verarbeite Binance Asset: {row['ticker']}")

def process_yahoof_asset(session, row, asset):
    data = load_yf_data(row, timeframe = '5m')

    # und in die Datenbank einfügen
    print(f"Verarbeite Yahoo Finance Asset: {row['ticker']}")
    insert_minute_bar(session, data, asset.id)
    
exchange_function_map = {
    'Binance': process_binance_asset,
    'Yahoo Finance': process_yahoof_asset
}

In [6]:
def load_assets_from_csv(file_path, session):
    try:
        assets_df = pd.read_csv(file_path)
        with open(file_path, 'r') as file:
            content = file.read()
            if not content.strip():
                print("Die CSV-Datei ist leer.")
                return
    except Exception as e:
        print(f"Fehler beim Einlesen der CSV-Datei: {e}")
        return
    
    for index, row in assets_df.iterrows():
        try:
            process_asset(session, row)
        except Exception as e:
            print(f"Fehler beim Verarbeiten des Assets {row['ticker']}: {e}")
            continue  # Fortfahren mit dem nächsten Asset



# Asset-Verarbeitung (Erstellen oder Aktualisieren)
def process_asset(session, row):
    # Prüfen, ob das Asset existiert
    asset = check_if_asset_exists(session, row['ticker'])
    
    if asset:
        print(f"Asset {row['ticker']} existiert bereits. Aktualisiere letzter Stand.")
        update_asset_last_update(session, row['ticker'], row['last_update'])
    else:
        print(f"Asset {row['ticker']} wird erstellt.")
        add_new_asset(session, row)
    
    # Verarbeiten des Assets basierend auf dem Exchange
    exchange = row['exchange']
    if exchange in exchange_function_map:
        exchange_function_map[exchange](session, row, asset)
    else:
        print(f"Unbekannter Exchange: {exchange}")


In [7]:
def main():
    # Datenbank-Verbindung und Session erstellen

    
    # CSV-Datei Pfad
    csv_file_path = 'ref_assets.csv'
    
    # Tabellen in der Datenbank erstellen
    #create_tables(engine)
    session= start_session()
    # Assets aus CSV laden und verarbeiten
    load_assets_from_csv(csv_file_path, session)

if __name__ == "__main__":
    main()



session started
Asset BTCUSD existiert bereits. Aktualisiere letzter Stand.
Asset BTCUSD wurde erfolgreich aktualisiert.
Fehler beim Verarbeiten des Assets BTCUSD: process_binance_asset() takes 2 positional arguments but 3 were given
Asset AAPL existiert bereits. Aktualisiere letzter Stand.
Asset AAPL wurde erfolgreich aktualisiert.
Asset: <Symbol(ticker=AAPL, name=Apple, market=stock)>
Verarbeite Yahoo Finance Asset: AAPL
Fehler beim Einfügen der MinuteBars für Symbol-ID 351: 'list' object has no attribute 'iterrows'
Asset EURUSD existiert bereits. Aktualisiere letzter Stand.
Asset EURUSD wurde erfolgreich aktualisiert.
Asset: <Symbol(ticker=EURUSD, name=Euro-Dollar, market=forex)>


$EURUSD: possibly delisted; no timezone found


Keine Daten für EURUSD im Zeitraum 2024-08-25 bis 2024-10-23
Verarbeite Yahoo Finance Asset: EURUSD
Fehler beim Einfügen der MinuteBars für Symbol-ID 352: 'NoneType' object has no attribute 'iterrows'
